# FACTS devices

# We implement the FACTS devices based on the following source:

    A. Panosyan, "Modeling of advanced power transmission system controllers",
    Ph.D. dissertation, Gottfried Wilhelm Leibniz Universität Hannover, 2010.

We reproduce the case study from the source using the following grid model:

![title](facts/facts_case_study_grid.png)

So far, we have implemented the following FACTS devices:
- Static Var Compensator (SVC)
- Static Synchronous Compensator (SSC) 
- Thyristor-Controlled Series Capacitor (TCSC)
- Voltage Source Converter Based High Voltage Direct Current (VSC-HVDC)

## Reference: case study grid

First, we import the required functions and set up plotting, the grid model and the data for the timeseries simulation.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from pandapower.test.loadflow.test_facts import facts_case_study_grid
from pandapower.timeseries import DFData, OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
from pandapower.pf.create_jacobian_facts import calc_y_svc_pu
from pandapower.run import runpp
from pandapower.create import (
    create_bus,
    create_buses,
    create_bus_dc,
    create_ssc,
    create_svc,
    create_vsc,
    create_tcsc,
    create_line_dc,
    create_line_from_parameters,
    create_line_dc_from_parameters,
    create_empty_network,
    create_load,
    create_ext_grid,
)
from pandapower.toolbox.grid_modification import drop_lines
from pandapower.plotting.simple_plot import simple_plot

In [ ]:
VOLTAGE_KV = "Voltage (kV)"
RES_BUS_VM_PU = "res_bus.vm_pu"
LEGEND_KWARGS = {"loc": "upper center", "bbox_to_anchor": (0.5, -0.15), "title": "Bus", "ncol": 3}
THYRISTOR_ANGLE = "Thyristor firing angle (degree)"

def plot_result(ax, result, title, ylabel, ylim=None, yticks=None, legend=False, legend_kwargs=None):
    result.plot(ax=ax, legend=False)
    if legend:
        kwargs = {} if legend_kwargs is None else legend_kwargs
        ax.legend(**kwargs)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel("Time (h)")
    ax.set_xscale('linear')
    locs = np.array([-24,   0,  24,  48,  72,  95])
    labels = ['00:00','00:00','06:00','12:00','18:00',"23:45"]
    ax.set_xticks(locs, labels)
    ax.set_xlim([0, 95])
    if ylim is not None:
        ax.set_ylim(ylim)
    if yticks is not None:
        ax.set_yticks(yticks)

In [ ]:
net = facts_case_study_grid()

All loads are considered constant. The output of the wind park is configured via a wind power profile. 
The plot below shows the wind power profile used in the timeseries simulation:

In [ ]:
#### define and prepare the generation and load profiles.

wind_profile = pd.read_csv(r"facts/facts_case_study_wind_profile.csv", index_col=0)

_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
plot_result(ax, wind_profile, "Wind power generation", "Power output (MW)")

In [ ]:
time_steps = wind_profile.index.values
wind_ds = DFData(wind_profile)
ConstControl(net, element='gen', variable='p_mw', element_index=net.gen.index,
             data_source=wind_ds, profile_name=["wind_power_mw"])

ow = OutputWriter(net, time_steps, output_path=None)
ow.log_variable("res_bus", "vm_pu")
ow.log_variable("res_trafo", "q_hv_mvar")
ow.log_variable("res_trafo", "q_lv_mvar")
ow.log_variable("res_gen", "p_mw")

In [ ]:
run_timeseries(net, time_steps, continue_on_divergence=False, recycle=False)

We do not include the buses "B1" and "B2" in the result plots because their voltage magnitude is always 1 p.u.
The results of the timeseries simulation for the bus voltages is are shown in the plot below:

In [ ]:
vm_pu = ow.output[RES_BUS_VM_PU].copy()
vm_pu.columns = net.bus.name.values

_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
plot_result(ax, 230 * vm_pu.iloc[:, 2:], "Bus voltage results", VOLTAGE_KV, legend=True, legend_kwargs=LEGEND_KWARGS)


In [ ]:
vm_pu.columns

## SVC use case

The element SVC is a shunt-connected thyristor-based power electronic device. It regulates the voltage magnitude at the
connected bus by adjusting the shunt impedance value. The shunt impedance only has the reactance component
(no resistive part). The impedance can be capacitive or inductive, because the device consists of a fixed capacitor in
a parallel circuit to the reactor that is regulated by a pair of antiparallel thyristors.
The thyristor firing angle regulates the total impedance of the element. The device operates in the value range of the thyristor firing angle between 90° and 180°.

The active range of the device is presented in the figure below:

In [ ]:
baseV = 230
baseMVA = 1
baseZ = baseV ** 2 / baseMVA

x = np.arange(90,180.1, 0.1)
y = calc_y_svc_pu(np.deg2rad(x), 1 / baseZ, -10 / baseZ)

_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(x, 1 / y * baseZ)
ax.set_ylim(-100, 100)
ax.set_xlabel(THYRISTOR_ANGLE)
ax.set_ylabel("SVC impedance (Ohm)")

It can be seen that the device has a resonance region
between the inductive (positive) and the capacitive (negative) impedance characteristics.
In real application, the operation of the device in the resonance region is prohibited.

Next, we create the SVC device connected to bus "B7" and configure it to regulate the voltage to the set point of 1 p.u.

In [ ]:
create_svc(net, bus=6, x_l_ohm=1, x_cvar_ohm=-10, set_vm_pu=1., thyristor_firing_angle_degree=90, controllable=True)
net.svc

In [ ]:
runpp(net)

The results of the power flow calculation for the SVC element are shown below: 

In [ ]:
net.res_svc

As you may noticed, the `thyrister_firing_angle_degree` in `net.res_svc` differs from the `net.svc` table. This is because `controllable` is True so that the value can be considered as starting value for the Newton Raphson algorithm. If we switch `controllable` to False and considere `thyrister_firing_angle_degree` next to the previous result, the new result will be next to the previous result but without adjusted `thyrister_firing_angle_degree` for voltage control.

In [ ]:
net.svc.at[0, "controllable"] = False
net.svc.at[0, "thyristor_firing_angle_degree"] = 143.625
runpp(net)
print(net.res_svc)

Next, we execute the timeseries simulation and plot the results (again with controlled voltage).

In [ ]:
net.svc.at[0, "controllable"] = True
ow.log_variable("res_svc", "thyristor_firing_angle_degree")
run_timeseries(net, time_steps=time_steps, continue_on_divergence=True, recycle=False)

In [ ]:
vm_pu_svc = ow.output[RES_BUS_VM_PU].copy()
vm_pu_svc.columns = net.bus.name.values

_, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
plot_result(ax1, 230 * vm_pu.iloc[:, 2:], "Bus voltage without SVC", VOLTAGE_KV,
            legend=True, legend_kwargs=LEGEND_KWARGS)
plot_result(ax2, 230 * vm_pu_svc.iloc[:, 2:], "Bus voltage with SVC", VOLTAGE_KV,
            legend=True, legend_kwargs=LEGEND_KWARGS)

The necessary thyristor firing angle to control the voltage is presented in the next plot.

In [ ]:
_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
thyristor_firing_angle =  ow.output["res_svc.thyristor_firing_angle_degree"].copy() 

plot_result(ax, thyristor_firing_angle, 
            "Thyristor firing angle of SVC", THYRISTOR_ANGLE)

# SSC use case (STATCOM)

The Static Synchronous Compensator (SSC), also known as STATCOM, is a shunt connected Flexible AC Transmission System
(FACTS) controller. It connects an AC system to a Voltage Source Converter (VSC) through a coupling transformer.
The SSC is used for reactive shunt compensation. Since the VSC is not connected to a power source, there is no active
power exchange between the SSC and the AC system. Consequently, the SSC can only control the voltage magnitude of
the AC system. At this use case, for the purpose of comparison, the SVC at bus B7 is replaced by a STATCOM to control the voltage amplitude to 230 kV.


In [ ]:
# net = facts_case_study_grid()
# putting svc out of serivce and replace it with ssc
net.svc.in_service = False

create_ssc(net, bus=6, r_ohm=0, x_ohm=5, set_vm_pu=1, controllable=True, in_service=True, vm_internal_pu=1.02, va_internal_degree=0)
# r_ohm, x_ohm are the parameters for the coupling transformer

runpp(net, init='flat')

In [ ]:
net.ssc

The results of the power flow calculation for the SSC element are shown below: 

In [ ]:
net.res_ssc

we can confirm that the voltage magnitude at the bus 'B7' which has the index 6,is maintained at 1 vm_pu buy checking bus voltages

In [ ]:
net.res_bus

Next, we execute the timeseries simulation and plot the results, in comparison to svc.

In [ ]:
ow.log_variable("res_ssc", "vm_internal_pu")

run_timeseries(net, time_steps=time_steps, continue_on_divergence=False, recycle=False,init='flat')

In [ ]:
vm_pu_ssc = ow.output[RES_BUS_VM_PU].copy()
vm_pu_ssc.columns = net.bus.name.values

_, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))
plot_result(ax1, 230 * vm_pu.iloc[:, 2:], "Bus voltage without SVC or SSC", VOLTAGE_KV,
            legend=True, legend_kwargs=LEGEND_KWARGS)
plot_result(ax2, 230 * vm_pu_svc.iloc[:, 2:], "Bus voltage with SVC", VOLTAGE_KV,
            legend=True, legend_kwargs=LEGEND_KWARGS)
plot_result(ax3, 230 * vm_pu_ssc.iloc[:, 2:], "Bus voltage with SSC", VOLTAGE_KV,
            legend=True, legend_kwargs=LEGEND_KWARGS)

Next, we compare the converter output voltage of the ssc device to the svc thyristor firing angle:

In [ ]:
vm_pu_ssc = ow.output["res_ssc.vm_internal_pu"].copy()
vm_pu_ssc
_, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))

plot_result(ax1, thyristor_firing_angle, 
            "Thyristor firing angle of SVC", THYRISTOR_ANGLE)

plot_result(ax2, vm_pu_ssc*230, 
            "Converter output voltage VSC", "Internal SSC Voltage (kV)")

## TCSC use case

In this use case, we connect a TCSC controllable element in series at the start of line "L4", then we execute a time series simulation when the TCSC element is active and when it is not active. To this end, we create an auxiliary bus and connect the TCSC element to it and reconnect the line "L4".

The approach is to adjust the structure of the grid before running the simulation.

Initial set-up:

    B4--------(Line 4)-------B6 

New set-up:

    B4--(TCSC)----aux------(Line 4)-------B6

We need to apply the following steps:

- Step 1: define the necessary parameters: $X_L$, $X_{C_{var}}$, thyristor firing angle and the active power set point.
- Step 2: add an auxiliary bus to connect the TCSC and reconnect the line.
- Step 3: identify the line 4 and reconnect it - adjust the data so that the line 4 is connected to the bus B4 as from_bus and aux as to_bus.
- Step 4: create the TCSC betwen B4 and the auxiliary bus, provide the required parameters and set controllable to True. This will activate the TCSC element.

Note: if controllable is set to False, TCSC will act as a fixed impedance corresponding to the parameters $X_L$ and  $X_{C_{var}}$.

In [ ]:
xl = 7.6
xc = -47

f = net.bus.loc[net.bus.name == "B4"].index.values[0]
t = net.bus.loc[net.bus.name == "B6"].index.values[0]
aux = create_bus(net, 230, "aux")
l = net.line.loc[(net.line.from_bus == f) & (net.line.to_bus == t)].index.values[0]
net.line.loc[l, "from_bus"] = aux

create_tcsc(net, from_bus=f, to_bus=aux, x_l_ohm=xl, x_cvar_ohm=xc, set_p_to_mw=-100,
               thyristor_firing_angle_degree=140, controllable=True)
net.svc.in_service = False
net.ssc.in_service = False

net.tcsc

In [ ]:
runpp(net, init="dc")
net.res_tcsc

Now, after loading the grid and creating TCSC element, we again set up the timeseries simulation. In the fololowing, we execute the first part of the calculation with the TCSC element with controllable set to True:

In [ ]:
ow.log_variable("res_tcsc", "thyristor_firing_angle_degree")
ow.log_variable("res_tcsc", "p_from_mw")
ow.log_variable("res_tcsc", "q_from_mvar")
ow.log_variable("res_tcsc", "ql_mvar")

run_timeseries(net, time_steps[:48], continue_on_divergence=True, recycle=False, max_iteration=100, init="results")
p_controllable = ow.output["res_tcsc.p_from_mw"].copy()
angle_controllable = ow.output["res_tcsc.thyristor_firing_angle_degree"].copy()

Now we set the TCSC parameter controllable to False and configure the thyristor firing angle to a fixed value of 90 degrees, then again execute the simulation. In the fololowing, we execute the second part of the calculation with the TCSC element acting as a fixed impedance:

In [ ]:
net.tcsc.controllable = False
net.tcsc.thyristor_firing_angle_degree = 90
run_timeseries(net, time_steps[48:], continue_on_divergence=True, recycle=False, max_iteration=100, init="results")
p_not_controllable = ow.output["res_tcsc.p_from_mw"].copy()
angle_not_controllable = ow.output["res_tcsc.thyristor_firing_angle_degree"].copy()
p_total = pd.concat([p_controllable, p_not_controllable])
angle_total = pd.concat([angle_controllable, angle_not_controllable])

Now we plot the active power transfer from buses 4 to 6 through the TCSC element and the line "L4" when TCSC is controllable (first half) and not controllable (second half):

In [ ]:
_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
plot_result(ax, p_total, "Active power flow through TCSC", "P (MW)", [-20, 180])

The corresponding thyristor firing angle is shown in the plot below:

In [ ]:
_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
plot_result(ax, angle_total, "Thyristor firing angle of TCSC", THYRISTOR_ANGLE)

# VSC-HVDC use case

The model of VSC-HVDC use case is now validated by replaceing one of the two cables connecting the wind farm to the
transmission system.A VSC-HVDC with 30km dc cable is employed between B3 and B4 instead of L2. nevertheless, connecting a DC cable between B3 and B4 cannot be done without converters (VSCs).Thus, it is required to create the DC system first then connect it with the AC system. in our case the DC system is consist of the 30km dc cable and two dc buses to be connected at the DC side of the VSCs. so the structure should look like the follwing

Initial set-up:

    B3--------(Line 2)-------B4 
    
The new set-up:
    
    B3-----VSC1---Bdc---------(30km dc line)--------Bdc----VSC2----B4

Now, lets import the grid in its intial set-up and apply the necessary adjustments.

In [ ]:
net = facts_case_study_grid()

First, we put line 2 out of service or we drop it

In [ ]:
drop_lines(net,lines=[1])

Now, we create dc buses and dc line(the 30 dc cable)

In [ ]:
create_bus_dc(net, 300, 'A')
create_bus_dc(net, 300, 'B')

create_line_dc(net,0,1,30,std_type='2400-AL')

Next, we create the VSCs, configure them and then connect the DC with the AC system.In our case, a single degree of freedom at each of the VSCs is employed to control the voltage magnitude at the two buses B3 and B4. The second degree of freedom of the rectifier side VSC2 (B3) is applied to control the active power flow either at 300 MW or at wind power level.

In [ ]:
create_vsc(net, bus=2, bus_dc=0, r_ohm=0, x_ohm=15, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac ="vm_pu", control_value_ac=1,
              control_mode_dc="p_mw", control_value_dc=-300)
#### TODO: explain the contro_value_dc being in negative 
create_vsc(net, bus=3, bus_dc=1, r_ohm=0, x_ohm=15, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac ="vm_pu", control_value_ac=1,
              control_mode_dc="vm_pu", control_value_dc=1)

Now, after loading the grid and creating VSC-HVDC element, we again set up the timeseries simulation.

In [ ]:
#### define and prepare the generation and load profiles.

wind_profile = pd.read_csv(r"facts/facts_case_study_wind_profile.csv", index_col=0)

wind_profile['vsc_set_p'] = -np.clip(wind_profile['wind_power_mw'],0,300)
wind_profile

In [ ]:
_, (ax) = plt.subplots(1, 1, figsize=(10, 5))
plot_result(ax, wind_profile, "Wind power generation", "Power output (MW)")
time_steps = wind_profile.index.values
wind_ds = DFData(wind_profile)
ConstControl(net, element='gen', variable='p_mw', element_index=net.gen.index,
             data_source=wind_ds, profile_name=["wind_power_mw"])
ConstControl(net, element='vsc', variable='control_value_dc', element_index=[0],
             data_source=wind_ds, profile_name=["vsc_set_p"])


ow = OutputWriter(net, time_steps, output_path=None)

ow.log_variable("res_vsc", "p_dc_mw")
ow.log_variable("res_vsc", "vm_internal_pu")
ow.log_variable("res_vsc", "va_internal_degree")
ow.log_variable("res_line", "p_from_mw")
ow.log_variable("res_gen", "p_mw")
ow.log_variable("res_line_dc","p_from_mw")
run_timeseries(net, time_steps=time_steps, continue_on_divergence=True, recycle=False,init='flat')

Now we plot the result for the power flow thorugh ac and dc lines in comparason with generated power

In [ ]:
line_ac_3_4 =  ow.output['res_line.p_from_mw'].copy()
line_dc_3_4 = ow.output['res_line_dc.p_from_mw'].copy()


_, (ax) = plt.subplots(1, 1, figsize=(10, 5))

results = pd.concat([wind_profile['wind_power_mw'],line_dc_3_4,line_ac_3_4[0]] ,axis=1)
results.columns = ['Wind generated active power','Active power flow through DC cable','Active power flow through AC line ']
plot_result(ax, results, 
            "Active power of VSC-HVDC", "P/[MW]", legend=True)

Now we plot the control variables at VSC-HVDC for both VSCs the rectifier and the inverter, which are the internal voltage magnitudes and Angles

In [ ]:
vsc_vm_internal_pu = ow.output['res_vsc.vm_internal_pu'].copy()
vsc_va_internal_degree = ow.output['res_vsc.va_internal_degree'].copy()

vsc_vm_internal_pu.columns = ['$V_r$','$V_i$']
vsc_va_internal_degree.columns = ['$\delta_r$','$\delta_i$']


_, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))

plot_result(ax1, vsc_vm_internal_pu*230, 
            "Voltage Magnitude Control Variables of VSC-HVDC", "Voltage Angle (kV)", legend=True)

plot_result(ax2, vsc_va_internal_degree, 
            "Voltage Anlges Control Variables of VSC-HVDC", "Voltage Angle (degree)", legend=True)


# Multi Terminal VSC-HVDC

In the previous example we connceted two terminal dc system to an ac system. but in fact multi terminal VSC-HVDC system can be formulated as we see in the following example

We first create the AC gird part

In [ ]:
net = create_empty_network()

create_buses(net, 5, 110, geodata=((0, 50), (50, 100), (200, 100), (50, 0), (200, 0)))
create_line_from_parameters(net, 0, 1, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 1, 2, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 0, 3, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 1, 3, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 3, 4, 30, 0.0487, 0.13823, 160, 0.664)
create_ext_grid(net, 0)
create_load(net, 2, 10)
create_load(net,4,15);

Now we plot the AC system, blue circles represnet AC buses and black lines represnets AC lines, the triangles represent loads.

In [ ]:
simple_plot(net, plot_loads=True, load_size=2);

Then we create the DC grid part

In [ ]:
create_bus_dc(net, 320, 'A', geodata=(50, 100))
create_bus_dc(net, 320, 'B', geodata=(200, 50))
create_bus_dc(net, 320, 'C', geodata=(200, 100))
create_bus_dc(net, 320, 'D', geodata=(200, 0))
create_bus_dc(net, 320, 'E', geodata=(50, 0))

create_line_dc_from_parameters(net, 0, 1, 100, 0.1, 1)
create_line_dc_from_parameters(net, 1, 2, 100, 0.1, 1)
create_line_dc_from_parameters(net, 1, 3, 100, 0.1, 1)
create_line_dc_from_parameters(net, 1, 4, 100, 0.1, 1);

Next we connect the AC grid with DC grid through VSCs

In [ ]:
create_vsc(net, bus=1, bus_dc=0, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="q_mvar",
              control_value_ac=3, control_mode_dc="vm_pu", control_value_dc=1)
create_vsc(net, bus=2, bus_dc=2, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="q_mvar",
              control_value_ac=10, control_mode_dc="p_mw", control_value_dc=5)
create_vsc(net, bus=4, bus_dc=3, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="vm_pu",
              control_value_ac=1.05, control_mode_dc="p_mw", control_value_dc=15)
create_vsc(net, bus=3, bus_dc=4, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="vm_pu",
              control_value_ac=1.03, control_mode_dc="vm_pu",control_value_dc=1.02);
net.vsc

Now, we plot both grids. The circles in blue color represent AC buses, the circles in purple color represent DC buses. Light blue squares shows VSCs, the light blue lines are the DC lines, and the triangles represent loads. 

In [ ]:
simple_plot(net, plot_loads=True, load_size=2);

Now, we present the power flow calculations.

In [ ]:
runpp(net)
net.res_line_dc

In [ ]:
net.res_line

In [ ]:
net.res_vsc

In [ ]:
net.res_bus_dc

In [ ]:
net.res_bus

# B2B VSC-HVDC

Now we try another configuration for VSC-HVDC as Back-to-Back(B2B). implementing this configuration
requires to connect 2 VSCs (at least) to the same DC Bus. One of the examples of B2B VSC-HVDC is Tres Amigas SuperStation, so let's build it.

In [ ]:
net = create_empty_network()
# AC part
create_buses(net, 5, 110, geodata=((0, 0), (70, -20), (100, 100), (100, -100), (200, 0)))
create_line_from_parameters(net, 0, 1, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 0, 2, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 0, 3, 30, 0.0487, 0.13823, 160, 0.664)
create_line_from_parameters(net, 1, 4, 30, 0.0487, 0.13823, 160, 0.664)
create_ext_grid(net, 0)
create_load(net, 2, 10)

# DC system
create_bus_dc(net, 110, 'A',geodata=(80, 20))

create_vsc(net, bus=4, bus_dc=0, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="vm_pu",
              control_value_ac=1, control_mode_dc="vm_pu", control_value_dc=1.02)
create_vsc(net, bus=2, bus_dc=0, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="vm_pu",
              control_value_ac=1, control_mode_dc="p_mw", control_value_dc=2)
create_vsc(net, bus=3, bus_dc=0, r_ohm=0.1, x_ohm=5, r_dc_ohm=0.5, pl_dc_mw=0.01, control_mode_ac="vm_pu",
              control_value_ac=1, control_mode_dc="p_mw", control_value_dc=4)

runpp(net)
simple_plot(net, bus_dc_size=3, vsc_size=4)

In [ ]:
net.res_vsc

In [ ]:
net.res_bus_dc

In [ ]:
net.res_bus